In [1]:
# Importando bibliotecas
from pyspark.sql import SparkSession
from pyspark.sql.functions import desc
import os
from warnings import filterwarnings
filterwarnings('ignore')

# Definindo variáveis de diretório
data_path = os.path.join(''.join(os.path.pardir + "/") * 3, 'data/flights-data/summary-data/parquet/2010-summary.parquet/part-r-00000-1a9822ba-b8fb-4d8e-844a-ea30d0801b9e.gz.parquet')

# Inicializando sessão
spark = (SparkSession
    .builder
    .appName("art09-spark-sql")
    .getOrCreate())
spark

22/07/26 21:24:38 WARN Utils: Your hostname, panini-ubuntu resolves to a loopback address: 127.0.1.1, but we couldn't find any external IP address!
22/07/26 21:24:38 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/07/26 21:24:39 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/07/26 21:24:41 WARN MacAddressUtil: Failed to find a usable hardware address from the network interfaces; using random bytes: b4:3f:00:99:ed:9c:13:82


In [2]:
# Criando database via SparkSQL (opcional)
spark.sql("CREATE DATABASE IF NOT EXISTS paninitlab")
spark.sql("USE paninitlab")

# Criando tabela
spark.sql("""
    CREATE TABLE IF NOT EXISTS flights_data (
        DEST_COUNTRY_NAME STRING,
        ORIGIN_COUNTRY_NAME STRING,
        count INT
    )
    
    USING csv OPTIONS (PATH '/home/hadoop/dev/panini-tech-lab/data/flights-data/summary-data/csv/2015-summary.csv')
""")



DataFrame[]

In [3]:
# Criando database via SparkSQL (opcional)
spark.sql("CREATE DATABASE IF NOT EXISTS paninitlab")
spark.sql("USE paninitlab")

# Dropando e criando tabela
spark.sql("DROP TABLE IF EXISTS flights_data")
spark.sql("""
    CREATE EXTERNAL TABLE IF NOT EXISTS flights_data (
        DEST_COUNTRY_NAME STRING,
        ORIGIN_COUNTRY_NAME STRING,
        count INT
    )

    -- Definindo formato de origem
    USING csv

    -- Definindo opções
    LOCATION '/home/hadoop/dev/panini-tech-lab/data/flights-data/summary-data/csv/2015-summary.csv'
    OPTIONS (
        header="true"
    )
""")

# Selecionando dados
df_flights_external = spark.sql("""
    SELECT * FROM flights_data LIMIT 5
""")
df_flights_external.show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+



In [4]:
# Lendo dados
csv_path = '/home/hadoop/dev/panini-tech-lab/data/flights-data/summary-data/csv/2015-summary.csv'
df_flights = spark.read.format("csv")\
    .option("inferSchema", "true")\
    .option("header", "true")\
    .load(csv_path)

# Criando tabela temporária (view)
df_flights.createOrReplaceTempView("vw_flights_data")

In [5]:
df_flights.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [6]:
# Consultando view
df_flights_view = spark.sql("""
    SELECT * FROM vw_flights_data LIMIT 5
""")

# Visualizando dados
df_flights_view.show()

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+



In [7]:
# Visualizando databases
spark.catalog.listDatabases()

# Visualizando tabelas
spark.catalog.listTables()

# Visualizando colunas de uma tabelas
spark.catalog.listColumns("flights_data")

[Column(name='DEST_COUNTRY_NAME', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='ORIGIN_COUNTRY_NAME', description=None, dataType='string', nullable=True, isPartition=False, isBucket=False),
 Column(name='count', description=None, dataType='int', nullable=True, isPartition=False, isBucket=False)]

In [8]:
# Visualizando tabelas
spark.catalog.listTables()

[Table(name='flights_data', database='paninitlab', description=None, tableType='EXTERNAL', isTemporary=False),
 Table(name='vw_flights_data', database=None, description=None, tableType='TEMPORARY', isTemporary=True)]

In [9]:
df_flights.show(5)

+-----------------+-------------------+-----+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME|count|
+-----------------+-------------------+-----+
|    United States|            Romania|   15|
|    United States|            Croatia|    1|
|    United States|            Ireland|  344|
|            Egypt|      United States|   15|
|    United States|              India|   62|
+-----------------+-------------------+-----+
only showing top 5 rows



In [10]:
# Principais destinos de vôos americanos
df_flights_eua_ordered = (df_flights.where("ORIGIN_COUNTRY_NAME = 'United States'")
                            .orderBy(desc("count")))
df_flights_eua_ordered.show(5)

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|           Canada|      United States|  8399|
|           Mexico|      United States|  7140|
|   United Kingdom|      United States|  2025|
|            Japan|      United States|  1548|
+-----------------+-------------------+------+
only showing top 5 rows



In [11]:
df_flights_eua_ordered.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#51 DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(count#51 DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [id=#187]
      +- Filter (isnotnull(ORIGIN_COUNTRY_NAME#50) AND (ORIGIN_COUNTRY_NAME#50 = United States))
         +- FileScan csv [DEST_COUNTRY_NAME#49,ORIGIN_COUNTRY_NAME#50,count#51] Batched: false, DataFilters: [isnotnull(ORIGIN_COUNTRY_NAME#50), (ORIGIN_COUNTRY_NAME#50 = United States)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/hadoop/dev/panini-tech-lab/data/flights-data/summary-data/c..., PartitionFilters: [], PushedFilters: [IsNotNull(ORIGIN_COUNTRY_NAME), EqualTo(ORIGIN_COUNTRY_NAME,United States)], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [12]:
# Principais destinos de vôos americanos (SparkSQL)
df_sql = spark.sql("""
    SELECT * FROM vw_flights_data
    WHERE ORIGIN_COUNTRY_NAME = 'United States'
    ORDER BY count DESC
""")

# Visualizando dados
df_sql.show(5)

+-----------------+-------------------+------+
|DEST_COUNTRY_NAME|ORIGIN_COUNTRY_NAME| count|
+-----------------+-------------------+------+
|    United States|      United States|370002|
|           Canada|      United States|  8399|
|           Mexico|      United States|  7140|
|   United Kingdom|      United States|  2025|
|            Japan|      United States|  1548|
+-----------------+-------------------+------+
only showing top 5 rows



In [13]:
# Visualizando plano de execução
df_sql.explain()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- Sort [count#51 DESC NULLS LAST], true, 0
   +- Exchange rangepartitioning(count#51 DESC NULLS LAST, 200), ENSURE_REQUIREMENTS, [id=#217]
      +- Filter (isnotnull(ORIGIN_COUNTRY_NAME#50) AND (ORIGIN_COUNTRY_NAME#50 = United States))
         +- FileScan csv [DEST_COUNTRY_NAME#49,ORIGIN_COUNTRY_NAME#50,count#51] Batched: false, DataFilters: [isnotnull(ORIGIN_COUNTRY_NAME#50), (ORIGIN_COUNTRY_NAME#50 = United States)], Format: CSV, Location: InMemoryFileIndex(1 paths)[file:/home/hadoop/dev/panini-tech-lab/data/flights-data/summary-data/c..., PartitionFilters: [], PushedFilters: [IsNotNull(ORIGIN_COUNTRY_NAME), EqualTo(ORIGIN_COUNTRY_NAME,United States)], ReadSchema: struct<DEST_COUNTRY_NAME:string,ORIGIN_COUNTRY_NAME:string,count:int>




In [14]:
df_flights.count()

256

In [15]:
# Vôos com origem e destino nos EUA
df1 = spark.sql("""
    SELECT
        ORIGIN_COUNTRY_NAME AS pais_origem,
        DEST_COUNTRY_NAME AS pais_destino,
        count AS qtd_voos

    FROM vw_flights_data

    WHERE ORIGIN_COUNTRY_NAME = 'United States'
        AND DEST_COUNTRY_NAME = 'United States'
""").show(5)

+-------------+-------------+--------+
|  pais_origem| pais_destino|qtd_voos|
+-------------+-------------+--------+
|United States|United States|  370002|
+-------------+-------------+--------+



In [16]:
# Quantidade de destinos
df2 = spark.sql("""
    SELECT 
        count(1) AS qtd_destinos

    FROM (
        SELECT
            DEST_COUNTRY_NAME
        
        FROM vw_flights_data

        WHERE ORIGIN_COUNTRY_NAME = 'United States'
            AND DEST_COUNTRY_NAME != 'United States'
    )
""").show()

+------------+
|qtd_destinos|
+------------+
|         131|
+------------+



In [18]:
df_flights.where("ORIGIN_COUNTRY_NAME = 'United States'").where("DEST_COUNTRY_NAME != 'United States'").count()

131

In [54]:
# Retornando top 5 países com maior diferença entre viagens de ida para os EUA do que viagens de volta
df3 = spark.sql("""
    WITH american_flights_destination AS (
        SELECT
            ORIGIN_COUNTRY_NAME,
            DEST_COUNTRY_NAME,
            count

        FROM vw_flights_data

        WHERE DEST_COUNTRY_NAME = 'United States'
            AND ORIGIN_COUNTRY_NAME != 'United States' 
    ),

    american_flights_origin AS (
        SELECT
            ORIGIN_COUNTRY_NAME,
            DEST_COUNTRY_NAME,
            count

        FROM vw_flights_data

        WHERE ORIGIN_COUNTRY_NAME = 'United States'
            AND DEST_COUNTRY_NAME != 'United States' 
    )

    -- Países com maior diferença entre viagens de ida e volta
    SELECT
        d.DEST_COUNTRY_NAME AS pais_destino,
        d.ORIGIN_COUNTRY_NAME AS pais_origem,
        o.count AS qtd_destino_para_origem,
        d.count AS qtd_origem_para_destino,
        (d.count - o.count) AS diff_ida_para_volta
    
    FROM american_flights_destination AS d

    INNER JOIN american_flights_origin AS o
        ON (d.ORIGIN_COUNTRY_NAME = o.DEST_COUNTRY_NAME)

    ORDER BY diff_ida_para_volta DESC

    LIMIT 5
""")

df3.show()

+-------------+------------------+-----------------------+-----------------------+-------------------+
| pais_destino|       pais_origem|qtd_destino_para_origem|qtd_origem_para_destino|diff_ida_para_volta|
+-------------+------------------+-----------------------+-----------------------+-------------------+
|United States|             China|                    772|                    920|                148|
|United States|            Canada|                   8399|                   8483|                 84|
|United States|         Hong Kong|                    332|                    414|                 82|
|United States|Dominican Republic|                   1353|                   1420|                 67|
|United States|              Peru|                    279|                    337|                 58|
+-------------+------------------+-----------------------+-----------------------+-------------------+

